In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

17:44:18 INFO:Extracting Amazon-Google...
17:44:18 INFO:Reading Amazon-Google row_dict...
17:44:18 INFO:Reading Amazon-Google train.csv...
17:44:18 INFO:Reading Amazon-Google valid.csv...
17:44:18 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'manufacturer', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

17:44:18 INFO:For attr=title, computing actual max_str_len
17:44:18 INFO:For attr=title, using actual_max_str_len=26
17:44:18 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:44:21 INFO:For attr=manufacturer, computing actual max_str_len
17:44:21 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:44:21 INFO:For attr=manufacturer, using actual_max_str_len=16
17:44:21 INFO:For attr=price, computing actual max_str_len
17:44:21 INFO:actual_max_str_len=9 must be pair to enable NN pooling. Updating to 10
17:44:21 INFO:For attr=price, using actual_max_str_len=10


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=26, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

17:44:22 INFO:GPU available: True, used: True
17:44:22 INFO:TPU available: None, using: 0 TPU cores
17:44:22 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

17:44:24 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.2 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
5.5 M     Trainable params
1.7 M     Non-trainable params
7.2 M     Total params


17:44:24 WARNING:true_pair_set has 82 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.39728397130966187,
 'semantic_title': 0.35876813530921936,
 'manufacturer': 0.12775567173957825,
 'price': 0.11619216203689575}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.4305427782888684,
 'valid_f1_at_0.5': 0.7508196721311476,
 'valid_f1_at_0.7': 0.8393574297188755,
 'valid_f1_at_0.9': 0.7164948453608248,
 'valid_pair_entity_ratio_at_0.3': 1.8665207877461707,
 'valid_pair_entity_ratio_at_0.5': 0.8227571115973742,
 'valid_pair_entity_ratio_at_0.7': 0.5776805251641138,
 'valid_pair_entity_ratio_at_0.9': 0.33698030634573306,
 'valid_precision_at_0.3': 0.27432590855803046,
 'valid_precision_at_0.5': 0.6090425531914894,
 'valid_precision_at_0.7': 0.7916666666666666,
 'valid_precision_at_0.9': 0.9025974025974026,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9786324786324786,
 'valid_recall_at_0.7': 0.8931623931623932,
 'valid_recall_at_0.9': 0.594017094017094}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.40381282495667237,
  'test_f1_at_0.5': 0.7252396166134185,
  'test_f1_at_0.7': 0.8286852589641434,
  'test_f1_at_0.9': 0.76,
  'test_pair_entity_ratio_at_0.3': 1.9956616052060738,
  'test_pair_entity_ratio_at_0.5': 0.8503253796095445,
  'test_pair_entity_ratio_at_0.7': 0.5813449023861171,
  'test_pair_entity_ratio_at_0.9': 0.3600867678958785,
  'test_precision_at_0.3': 0.2532608695652174,
  'test_precision_at_0.5': 0.5790816326530612,
  'test_precision_at_0.7': 0.7761194029850746,
  'test_precision_at_0.9': 0.9156626506024096,
  'test_recall_at_0.3': 0.9957264957264957,
  'test_recall_at_0.5': 0.9700854700854701,
  'test_recall_at_0.7': 0.8888888888888888,
  'test_recall_at_0.9': 0.6495726495726496}]